In [2]:
from sklearn import tree
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
#список пользователей по leader-id  и email из ple
users=pd.read_csv('C:/Users/d.muravlyansky/Documents/UNTI/Архипелаг/ple_user.csv', \
                              sep=';',encoding='utf-8')

# zoom - пока загружаем только тех у кого уникальный e-mail.
zoom=pd.read_csv('C:/Users/d.muravlyansky/Documents/UNTI/Архипелаг/zoom_list_13_11_2020.csv', \
                              sep=';',encoding='utf-8')

#Статистика вовлеченности за день
stat=pd.read_csv('C:/Users/d.muravlyansky/Documents/UNTI/Архипелаг/engagement_by_day_v15_13_11_2020.csv', \
                              sep=';',encoding='utf-8')


,untiID,leaderID,День,Загружена в PT презентация по проекту,Проект в ПТ размечен,Участником создана задача в Командном профиле,Выполнена задача в ком.профиле,Верифицировано решение в ком.профиле,Создание Темы изучения в Ком.профиле,Закрыта Тема изучения в Ком.профиле,...,Запись на повышение эффективности,Запись на Питч-сессию,Отредактирована цель,Оценено продвижение к цели в ЛК,Оставлен Фидбек на мероприятие (хотя бы один ответ),Создан Продукт в Edumap,Поставлен лайк проекту в ПТ,Поставлен лайк продукту (карточке) в Edumap,Создано вакансий,Откликов на вакансии
0,14960,69,2020-11-07,1,1,0,0,0,0,0,...,0,0,0,0,1,0,1,0,2,0
1,14960,69,2020-11-08,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,14960,69,2020-11-09,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,14960,69,2020-11-10,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,14960,69,2020-11-11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,14960,69,2020-11-12,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
6,14960,69,2020-11-13,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,25474,87,2020-11-07,1,1,0,0,0,0,0,...,1,0,0,0,2,0,1,0,1,0
8,25474,87,2020-11-08,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
9,25474,87,2020-11-09,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# Сшиваем по полю email
zoom_merged = pd.merge(
    zoom,
    users,
    on=['email'],
    how='left' ).fillna(0)

zoom_merged.leaderID=zoom_merged.leaderID.astype('int')
zoom_merged['zoom_name_leaderID']=zoom_merged.user_name.str.extract(r'\D*(\d*)\D*')
zoom_merged['zoom_name_leaderID']=zoom_merged['zoom_name_leaderID'].str.lstrip('0')
zoom_merged['zoom_name_leaderID']= pd.to_numeric(zoom_merged['zoom_name_leaderID'],downcast='integer').fillna(0).astype('int')

zoom_merged=zoom_merged.rename(columns={ 'leaderID': 'email_leaderID'})
zoom_merged.head(10)

,user_name,email,День,zoom_count,profile,email_leaderID,zoom_name_leaderID
0,᠌ ᠌᠌ ᠌ ᠌᠌ ᠌ ᠌ ᠌ ᠌ ᠌ ᠌᠌ ᠌ ᠌᠌ ᠌ ᠌ ᠌ ᠌,bravo0987@bk.ru,2020-11-07,2,devnull1578401@uni2035,1578401,0
1,???????? Alexei Sorokin,plannerix@gmail.com,2020-11-07,4,0,0,0
2,����� ��������,0,2020-11-07,1,0,0,0
3,0000087 Vorobyev Anton,anton@finexcons.ru,2020-11-07,10,anton5@uni2035,87,87
4,0022151 Efremov Dmitry,dmitry.efremov@gmail.com,2020-11-07,18,0,0,22151
5,0023817 Erman Mikhail,mikhail.erman@gmail.com,2020-11-07,2,0,0,23817
6,0046020 Daurkin Alexander,adaurkin@rambler.ru,2020-11-07,36,adaurkin@uni2035,46020,46020
7,0072723 Egorov Semen,xlo.ykt@gmail.com,2020-11-07,12,xloykt@uni2035,72723,72723
8,0344402 Struchkov Michil,michilstrvic@gmail.com,2020-11-07,12,michilstrvic@uni2035,344402,344402
9,0359328 Nurgun Struchkov,vokhcurts@mail.ru,2020-11-07,6,vokhcurts@uni2035,359328,359328


In [19]:
#Логика текущая такая 
#1. если есть в имени номер - используем его
#2. если в имени номера нет - берем leader_id
#3.(пока не сделано) если записей несколько (разные email), то сначала берем номер, а если его нет - любой из найденных leaderID
# для п.3 надо менять в частности и sql часть

condlist = [zoom_merged.zoom_name_leaderID==0, zoom_merged.zoom_name_leaderID>0]
choicelist = [zoom_merged.email_leaderID, zoom_merged.zoom_name_leaderID]
zoom_merged['leaderID']= np.select(condlist,choicelist)

zoom_merged.head(3)

,user_name,email,День,zoom_count,profile,email_leaderID,zoom_name_leaderID,leaderID
0,᠌ ᠌᠌ ᠌ ᠌᠌ ᠌ ᠌ ᠌ ᠌ ᠌ ᠌᠌ ᠌ ᠌᠌ ᠌ ᠌ ᠌ ᠌,bravo0987@bk.ru,2020-11-07,2,devnull1578401@uni2035,1578401,0,1578401
1,???????? Alexei Sorokin,plannerix@gmail.com,2020-11-07,4,0,0,0,0
2,����� ��������,0,2020-11-07,1,0,0,0,0


In [36]:
#Так как имя может писаться по разному надо перегруппировать теперь  по leaderID  строки
#
zoom_regrouped= zoom_merged[zoom_merged.leaderID>0].groupby(['leaderID','День'], as_index=True). \
    agg(zoom_count=('zoom_count', 'sum')).fillna(0)

zoom_regrouped.head(5)

zoom_count
leaderID День                  
1        2020-11-07          35
         2020-11-08          11
         2020-11-09           3
         2020-11-10           4
         2020-11-11           3

In [37]:
# Сшиваем по полям leaderID и День
stat_merged = pd.merge(
    stat,
    zoom_regrouped,
    on=['leaderID','День'],
    how='outer' ).fillna(0)
stat_merged.zoom_count=stat_merged.zoom_count.astype('int')

stat_merged.head(10)

,untiID,leaderID,День,Загружена в PT презентация по проекту,Проект в ПТ размечен,Участником создана задача в Командном профиле,Выполнена задача в ком.профиле,Верифицировано решение в ком.профиле,Создание Темы изучения в Ком.профиле,Закрыта Тема изучения в Ком.профиле,...,Запись на Питч-сессию,Отредактирована цель,Оценено продвижение к цели в ЛК,Оставлен Фидбек на мероприятие (хотя бы один ответ),Создан Продукт в Edumap,Поставлен лайк проекту в ПТ,Поставлен лайк продукту (карточке) в Edumap,Создано вакансий,Откликов на вакансии,zoom_count
0,14960.0,69,2020-11-07,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,2
1,14960.0,69,2020-11-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,8
2,14960.0,69,2020-11-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
3,14960.0,69,2020-11-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,13
4,14960.0,69,2020-11-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15
5,14960.0,69,2020-11-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,4
6,14960.0,69,2020-11-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
7,25474.0,87,2020-11-07,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0,14
8,25474.0,87,2020-11-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19
9,25474.0,87,2020-11-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9


In [39]:
#Запишем обратно в файл 
stat_merged.rename(columns={ 'zoom_count': 'Созвонов в Zoom'}). \
to_csv('C:/Users/d.muravlyansky/Documents/UNTI/Архипелаг/engagement_by_day_v15_13_11_2020_fin.csv', encoding='ANSI')  